In [70]:
import math
import numpy as np
import pandas as pd
N = 0
START = 0.1
STOP = 1
EPS = 0.1
def function(x):
  global N
  N += 1
  return 10*x*math.log(x) - (x**2 / 2)

In [71]:
import sympy as sp
x = sp.Symbol('x')
y = 10*x*sp.ln(x) - (x**2 / 2)
print(y.diff(x))
print(y.diff(x, x))

-x + 10*log(x) + 10
-1 + 10/x


In [72]:
def first_diff(x):
  global N
  N += 1
  return -x+10*math.log(x)+10

def second_diff(x):
  global N
  N += 1
  return -1 + 10/x

In [73]:
EPSILONS = [1e-1, 1e-2, 1e-3, 1e-4, 1e-5, 1e-6]
funcs = []

In [74]:
#Перебор
def perebor(start, stop, epsilon):
  numberOfSteps = int((stop-start)/epsilon)
  minF = function(start)
  minArg = start
  for val in np.linspace(start, stop, num=numberOfSteps):
    eval = function(val)
    if (minF > eval):
      minF = eval
      minArg = val
  return (minF, minArg)
funcs.append(('Перебор', perebor))

In [75]:
# Поразрядный поиск
def porPoisk(start, stop, epsilon):
  step = (stop - start)/4
  if step < epsilon:
    print(f'step > epsilon, step={step} epsilon={epsilon}')
  minArg = val = start
  fNext = function(minArg)
  minF = 0
  while abs(step) > epsilon:
    val += step
    f = fNext
    fNext = function(val)

    if fNext > f:
      minF = f;
      minArg = val
      step /= -4
      # print('change direction')
  return (minF, minArg)
funcs.append(('Поразрядный поиск', porPoisk))
  

In [76]:
# Дихотомия
def dihotomia(start, stop, epsilon, delta):
  if (0 > delta) or (delta > 2 * epsilon):
    print('invalid q')
    return 0
  leftPoint = (start + stop - delta) / 2
  rightPoint = (start + stop + delta) / 2
  while (stop - start)/2 > epsilon:

    if function(leftPoint) < function(rightPoint):
      stop = rightPoint
    else:
      start = leftPoint
    leftPoint = (start + stop - delta) / 2
    rightPoint = (start + stop + delta) / 2
  minArg = (start + stop) / 2
  minF = function(minArg)
  return (minF, minArg)


def normalizer(start, stop, epsilon):
  return dihotomia(start, stop, epsilon, epsilon)

  
funcs.append(('Дихотомия', normalizer))

In [77]:
# Золотое сечение
def golden(start, stop, epsilon):
  tau = (math.sqrt(5) - 1) / 2
  leftPoint = start + (1 - tau)*(stop - start)
  rightPoint = start + tau*(stop - start)
  leftF = function(leftPoint)
  rightF = function(rightPoint)
  while abs(stop - start) > epsilon:
    if leftF >= rightF:
      start = leftPoint
      leftPoint = rightPoint
      leftF = rightF
      rightPoint = start + tau*(stop - start)
      rightF = function(rightPoint)
    else:
      stop = rightPoint
      rightPoint = leftPoint
      rightF = leftF
      leftPoint = start + (1 - tau)*(stop - start)
      leftF = function(leftPoint)
  minArg = (start + stop) / 2
  minF = function(minArg)
  return (minF, minArg)
funcs.append(('Золотое сечение', golden))
    

In [78]:
def parabola(start, stop, epsilon):
  (x1, x2, x3, f1, f2, f3) = goldPoints(start, stop)

  a1 = (f2 - f1)/(x2 - x1)
  a2 = 1/(x3 - x2) * ((f3 - f1)/(x3 - x1) - (f2 - f1)/(x2 - x1))
  newX = (x1 + x2 - a1/a2)/2

  if (x1<newX<x2<x3):
    x1 = newX
    f1 = function(newX)
  
  if ((x1<x2<newX<x3)):
    x3 = newX
    f3 = function(newX)

  oldX = newX

  a1 = (f2 - f1)/(x2 - x1)
  a2 = 1/(x3 - x2) * ((f3 - f1)/(x3 - x1) - (f2 - f1)/(x2 - x1))
  newX = (x1 + x2 - a1/a2)/2

  while abs(oldX - newX) > epsilon:

    if (x1<newX<x2<x3):
      x1 = newX
      f1 = function(newX)
  
    if ((x1<x2<newX<x3)):
      x3 = newX
      f3 = function(newX)

    oldX = newX

    a1 = (f2 - f1)/(x2 - x1)
    a2 = 1/(x3 - x2) * ((f3 - f1)/(x3 - x1) - (f2 - f1)/(x2 - x1))
    newX = (x1 + x2 - a1/a2)/2
  
  minArg = newX
  minF = function(newX)
  return (minF, minArg)

def goldPoints(start, stop):
  tau = (math.sqrt(5) - 1) / 2
  leftPoint = start + (1 - tau)*(stop - start)
  rightPoint = start + tau*(stop - start)
  leftF = function(leftPoint)
  rightF = function(rightPoint)
  startF = function(start)
  stopF = function(stop)
  while True:

    if (startF >= leftF < stopF) or (startF > leftF <= stopF):
      return (start, leftPoint, stop, startF, leftF, stopF)

    if (leftF >= rightF < stopF) or (leftF > rightF <= stopF):
      return (leftPoint, rightPoint, stop, leftF, rightF, stopF)

    if leftF >= rightF:
      start = leftPoint
      leftPoint = rightPoint
      startF = leftF
      leftF = rightF
      rightPoint = start + tau*(stop - start)
      rightF = function(rightPoint)
    else:
      stop = rightPoint
      rightPoint = leftPoint
      stopF = rightF
      rightF = leftF
      leftPoint = start + (1 - tau)*(stop - start)
      leftF = function(leftPoint)
funcs.append(('Парабола', parabola))

In [79]:
# Средняя точка
def meanPoint(start, stop, epsilon):
  mP = (start + stop)/2
  fD = first_diff(mP)
  while (abs(fD) > epsilon):
    if (fD > 0):
      stop = mP

    if (fD < 0):
      start = mP
    
    mP = (start + stop)/2
    fD = first_diff(mP)
  
  minArg = mP
  minF = function(mP)
  return (minF, minArg)  

funcs.append(('Средняя точка', meanPoint))

In [80]:
# Метод хорд
def horda(start, stop, epsilon):
  startDiff = first_diff(start)
  stopDiff = first_diff(stop)
  hP = start - startDiff/(startDiff - stopDiff) * (start - stop)
  tmpDiff = first_diff(hP)
  while abs(tmpDiff) > epsilon:
    if (tmpDiff < 0):
      start = hP
      startDiff = first_diff(start)

    if (tmpDiff > 0):
      stop = hP
      stopDiff = first_diff(stop)

    hP = start - startDiff/(startDiff - stopDiff) * (start - stop)
    tmpDiff = first_diff(hP)

  minArg = hP
  minF = function(hP)
  return (minF, minArg)  

funcs.append(('Метод Хорд', horda))

In [81]:
# Ньютон
def newton(start, stop, epsilon):
  xprev = start
  fD = first_diff(xprev)
  sD = second_diff(xprev)
  nP = xprev - fD/sD

  while abs(fD) > epsilon:
    xprev = nP
    fD = first_diff(xprev)
    sD = second_diff(xprev)
    nP = xprev - fD/sD

  minArg = nP
  minF = function(nP)
  return (minF, minArg)  

funcs.append(('Метод Ньютона', newton))

In [82]:
newDf = pd.DataFrame(columns=["name","minArg", "minFunction", "N", 'EPS'])
for eps in EPSILONS:
  for f in funcs:
    (minF, minArg) = f[1](START, STOP, eps)
    newDf = pd.concat([newDf, pd.Series({"name": f[0],"minArg": minArg, "minFunction": minF, "N": N, "EPS": eps}).to_frame().T], ignore_index=True)
    N = 0

In [83]:
newDf

,name,minArg,minFunction,N,EPS
0,Перебор,0.4375,-3.712422,10,0.1
1,Поразрядный поиск,0.55,-3.705585,3,0.1
2,Дихотомия,0.375,-3.748422,9,0.1
3,Золотое сечение,0.403193,-3.743645,8,0.1
4,Парабола,0.400677,-3.744861,6,0.1
5,Средняя точка,0.38125,-3.749069,5,0.1
6,Метод Хорд,0.384303,-3.749026,16,0.1
7,Метод Ньютона,0.382208,-3.749081,9,0.1
8,Перебор,0.383146,-3.74907,91,0.01
9,Поразрядный поиск,0.395312,-3.749069,12,0.01


In [84]:
#newDf.groupby('EPS').apply(print)
newDf.sort_values(by=['name', 'EPS'], ascending=[True, False])#.apply(lambda a: a[:])

,name,minArg,minFunction,N,EPS
2,Дихотомия,0.375,-3.748422,9,0.1
10,Дихотомия,0.379648,-3.748998,15,0.01
18,Дихотомия,0.381876,-3.74908,21,0.001
26,Дихотомия,0.38223,-3.749081,29,0.0001
34,Дихотомия,0.38221,-3.749081,35,0.00001
42,Дихотомия,0.382213,-3.749081,41,0.000001
3,Золотое сечение,0.403193,-3.743645,8,0.1
11,Золотое сечение,0.382638,-3.749079,13,0.01
19,Золотое сечение,0.382104,-3.749081,18,0.001
27,Золотое сечение,0.38223,-3.749081,22,0.0001
